### What is vector seach?

Vector search leverages machine learning (ML) to capture the meaning and context of unstructured data, including text and images, transforming it into a numeric representation. Frequently used for semantic search, vector search finds similar data using approximate nearest neighbor (ANN) algorithms. Compared to traditional keyword search, vector search yields more relevant results and executes faster.

`semantic search`
 is a search method that focuses on understanding the meaning and intent behind a user's search query, rather than just matching keywords.

`Vector search` is a technique where text (or images, code, etc.) is converted into numerical vectors, and search is done by comparing the similarity of vectors — instead of matching words.

```It lets you search by meaning, not just keywords.```



### What is an Embedding?

To apply `vector search` we need to embed the unstructured data we want to perform our vector search on.

An `embedding` is just a list of numbers that represents the meaning of your text.

for example

``` "The cat is cute" → [0.12, -0.44, 0.91, ..., 0.03] ```

### What is an Embedding Model?
An embedding model is a special AI model that creates these embeddings.

It takes text like this:


``` "The cat is cute" ```

And turns it into something like this:

``` [0.12, -0.44, 0.91, ..., 0.03] ```  

You can think of the embedding model as:

A translator that turns words into vectors (math) that computers can search, compare, and rank.


After vectorizing our data, we can perform operations like similarity search. Tools like Qdrant help by storing and indexing these vectors for fast and efficient retrieval.

### What is Qdrant?

Qdrant “is a vector similarity search engine that provides a production-ready service with a convenient API to store, search, and manage points (i.e. vectors) with an additional payload.” You can think of the payloads as additional pieces of information that can help you hone in on your search and also receive useful information that you can give to your users.

Qdrant’s Role:

- Qdrant is a vector database.

    It can:

    ✅ Store vectors (and associated metadata, aka “payload”)

    ✅ Index them efficiently

    ✅ Search them by similarity (e.g. cosine similarity)

    ✅ Filter results using metadata (e.g. filter by language, category, etc.)



### Qdrant Vs Elasticsearch

`Qdrant` and `Elasticsearch` differ in how they index and search data:

`Elasticsearch` is optimized for keyword-based search, using inverted indexes to match exact terms or phrases.

`Qdrant`, on the other hand, is designed for semantic search, using vector indexes to find similar meanings rather than exact words.

#### `Even Simpler Version`:

`Elasticsearch` finds documents by matching words.
`Qdrant` finds documents by matching meaning.`

### Let's start working with qdrant.

`quickstart`

- Use the following link to quickly set up you Qdrant.

https://qdrant.tech/documentation/quickstart/

### Set up python client 

- install required library

        ```pip install -q "qdrant-client[fastembed]>=1.14.2"```

- It provides a client interface so your Python code can easily talk to the Qdrant server via its API.
